In [181]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df = melb_df.drop('YearBuilt', axis=1)

melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,AgeBuilding
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067,...,1,1,202.0,126.0,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,46
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067,...,1,0,156.0,79.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,116
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,3067,...,2,0,134.0,150.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,117
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,2017-03-04,2.5,3067,...,2,1,94.0,126.0,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,47
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,3067,...,1,2,120.0,142.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,2


In [182]:
# Задание 2.4

countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})

countries_df['density'] = (countries_df['population']) * 1000000/ countries_df['square']
round(countries_df['density'].mean(), 2)

84.93

In [183]:
# Задание 3.3

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [184]:
UFO_df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep=',')
display(UFO_df)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45


In [185]:
# Задание 3.4

UFO_df['Time'] = pd.to_datetime(UFO_df['Time'], dayfirst=False)
year_UFO = UFO_df['Time'].dt.year
print(year_UFO.mode())

0    1999
Name: Time, dtype: int32


In [186]:
# Задание 3.5

UFO_df['Date'] = UFO_df['Time'].dt.date
# Метод diff() вычисляет разницу между двумя соседними датами в столбце.
# Маска по штату 'NV', в которой мы применяет diff() по 'Date', а дальше вытаскиваем кол-во дней и находим среднее.
print(round(UFO_df[UFO_df['State']=='NV']['Date'].diff().dt.days.mean()))

69


In [187]:
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type)

popular_stypes = street_types.value_counts().nlargest(10).index

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')

melb_df = melb_df.drop('Address', axis=1)

In [188]:
# Задание 4.2

def get_weekend(weekday):
    if (weekday == 5) or (weekday == 6):
        return 1
    else:
        return 0
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

print(round(melb_df[melb_df['Weekend'] == 1]['Price'].mean()))

1081199


In [189]:
# Задание 4.3

popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')

a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))

1.3


In [190]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
max_unique_count = 150 
for col in melb_df.columns:
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude:
        melb_df[col] = melb_df[col].astype('category')
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  category      
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  category      
 5   SellerG        13580 non-null  category      
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  CouncilArea    12211 non-null  category      
 15  Lattitude      1358

None

In [191]:
# Задание 5.1

popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')

melb_df['Suburb'] = melb_df['Suburb'].astype('category')
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  category      
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  category      
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  category      
 5   SellerG        13580 non-null  category      
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  CouncilArea    12211 non-null  category      
 15  Lattitude      1358

None

In [192]:
bike_data = pd.read_csv('data/citibike-tripdata.csv', sep=',')
bike_df = bike_data.copy()

In [193]:
# Задание 6.1

display(bike_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

None

In [194]:
# Задание 6.3

print(bike_df['start station id'].mode()[0])

281.0


In [195]:
# Задание 6.4

print(bike_df['bikeid'].mode()[0])

33887


In [196]:
# Задание 6.5

usertype_df = bike_df['usertype'].value_counts().to_frame()
subs = usertype_df.loc['Subscriber', 'count']
custs = usertype_df.loc['Customer', 'count']
print(round(subs / (subs + custs), 2))


mode_usertype = bike_df['usertype'].mode()[0]
count_mode_user = bike_df[bike_df['usertype'] == mode_usertype].shape[0]
print(round(count_mode_user / bike_df.shape[0], 2))

0.77
0.77


In [197]:
# Задание 6.6

pop_gender = bike_df['gender'].value_counts()[1]
print(pop_gender)

183582


In [198]:
# Задание 6.7

print(bike_df['start station id'].nunique())
print(bike_df['end station id'].nunique())


def age(year):
    age = 2024 - year 
    return age

display(bike_df['birth year'].apply(age).min())


display(bike_df['start station name'].value_counts().index[-1])


display(bike_df['end station name'].value_counts().index[0])

759
765


22

'Eastern Pkwy & Washington Ave'

'West St & Chambers St'

In [199]:
# Задание 6.8

bike_df = bike_df.drop(['start station id', 'end station id'], axis=1)
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station name       299831 non-null  object 
 3   start station latitude   300000 non-null  float64
 4   start station longitude  300000 non-null  float64
 5   end station name         299831 non-null  object 
 6   end station latitude     300000 non-null  float64
 7   end station longitude    300000 non-null  float64
 8   bikeid                   300000 non-null  int64  
 9   usertype                 300000 non-null  object 
 10  birth year               300000 non-null  int64  
 11  gender                   300000 non-null  int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 27.5+ MB


In [200]:
# Задание 6.9

bike_df['age'] = 2018 - bike_df['birth year']
bike_df = bike_df.drop('birth year', axis=1)
bike_df[bike_df['age'] > 60].shape[0]

11837

In [205]:
# Задание 6.10

bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])
bike_df['trip duration'] = bike_df['stoptime'] - bike_df['starttime']
display(bike_df['trip duration'][3])

Timedelta('0 days 00:07:16.837000')

In [211]:
# Задание 6.11

# Из задания 4.2:
# def get_weekend(weekday):
#    if (weekday == 5) or (weekday == 6):
#        return 1
#    else:
#        return 0

bike_weekday = bike_df['starttime'].dt.dayofweek
bike_df['weekend'] = bike_weekday.apply(get_weekend)
print(bike_df[bike_df['weekend'] == 1].shape[0])

115135


In [212]:
# Задание 6.12

def get_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    elif 6 < time <= 12:
        return 'morning'
    elif 12 < time <= 18:
        return 'day'
    elif 18 < time <= 23:
        return 'evening'

bike_df['time_of_day'] = bike_df['starttime'].dt.hour.apply(get_time_of_day)
day_ride = bike_df[bike_df['time_of_day'] == 'day'].shape[0]
night_ride = bike_df[bike_df['time_of_day'] == 'night'].shape[0]
print(round(day_ride / night_ride))

9
